# Chi-Square Fit Test (Descrete)

In [116]:
import random
from scipy import stats

In [117]:
def cs_compute_t(n_i_list, h0_function):
    n = sum(n_i_list)
    
    # compute T
    t = 0
    for i, n_i in enumerate(n_i_list):
        n_p_i = n * h0_function(i)
        t += ((n_i - n_p_i)**2) / n_p_i
    
    return t

In [118]:
def cs_compute_pvalue(t, k):
    return 1 - stats.chi2.cdf(t, k - 1)

In [119]:
def cs_is_hypothesis_accepted(pvalue, threshold=0.05):
    return pvalue > threshold

### Chi-Square Example

In [120]:
def cs_example():
    n_i_list = [12, 5, 19, 7, 7] # n_i_list[i]=x shows that i'th sample was repeated x times in simulation
    h0_function = lambda i: 0.2

    t = cs_compute_t(n_i_list, h0_function)
    pvalue = cs_compute_pvalue(t, len(n_i_list))
    is_accepted = cs_is_hypothesis_accepted(pvalue)
    
    if is_accepted:
        print(f'hypothesis was accepted with pvalue: {pvalue} and t: {t}')
    else:
        print(f'hypothesis was not accepted with pvalue: {pvalue} and t: {t}')

cs_example()

hypothesis was not accepted with pvalue: 0.012295523821487131 and t: 12.8


___

# Kolmogrov-Smronov Fit Test (Continuous)

In [121]:
import math
import random

In [122]:
def ks_compute_d(sim_samples, h0_function):
    n = len(sim_samples)
    
    # 1. sort simulation values (y)
    sim_samples.sort()
    y_list = sim_samples
    
    # 2. compute (j/n) - H0(y[j]) for all js
    f_fe_list = [((j/n) - h0_function(y_list[j-1])) for j in range(1, n+1)]
    
    # 3. compute H0(y[j]) - ((j-1)/n) for all js
    fe_f_list = [(h0_function(y_list[j-1]) - ((j-1)/n)) for j in range(1, n+1)]
    
    # 4. return d as maximum of step 2 and 3 results
    return max([max(f_fe_list), max(fe_f_list)])    

In [123]:
def ks_compute_pvalue(d, n, iterations=1000):
    p = 0
    
    for i in range(iterations):
        # generate n uniform random variables
        u_list = [random.random() for j in range(n)]
        u_list.sort()
        
        # generate f_fe and fe_f list for these u list
        f_fe_list = [((j/n) - u_list[j-1]) for j in range(1, n+1)]
        fe_f_list = [(u_list[j-1] - ((j-1)/n)) for j in range(1, n+1)]
        
        # add number of items that are more than d in the preceding lists
        p += len([x for x in f_fe_list if x >= d])
        p += len([x for x in fe_f_list if x >= d])
    
    return p/iterations

In [124]:
def ks_is_hypothesis_accepted(pvalue, threshold=0.05):
    return pvalue > threshold

### Kolmogrov-Smronov Example

In [125]:
def ks_example():
    sim_samples = [66, 72, 81, 94, 112, 116, 124, 140, 145, 155]
    h0_function = lambda x: (1 - math.exp((-x)/100))

    d = ks_compute_d(sim_samples, h0_function)
    pvalue = ks_compute_pvalue(d, 10)
    is_accepted = ks_is_hypothesis_accepted(pvalue)

    if is_accepted:
        print(f'hypothesis was accepted with pvalue: {pvalue}')
    else:
        print(f'hypothesis was not accepted with pvalue: {pvalue}')

ks_example()

hypothesis was not accepted with pvalue: 0.013


___